In [ ]:
import requests
import re
import json

cache = {}
def hashable_cache(f):
    def inner(url, session):
        if url not in cache:
            cache[url] = f(url, session)
        return cache[url]
    return inner


from functools import lru_cache
@hashable_cache
def get_first_paragraph(wikipedia_url,session):
    leader=session.get(wikipedia_url)
    #html parser
    from bs4 import BeautifulSoup
    soup=BeautifulSoup(leader.text,'html.parser')
    #get first paragraphs
    paragraphs=[]
    first_paragraph=""
    break_flag=False
    for paragraph in soup.find_all("p"):
        for para in paragraph:
            check=str(para)
            if check.startswith("<b>"):
                first_paragraph=paragraph.text
                break_flag=True
                break
        if break_flag:
            break        
    #cleaning using regex
    #print(first_paragraph)
    pattern="\s*\[[^\]\s]*\]\s*"
    string=first_paragraph
    #print(re.findall(pattern,string))
    clean=re.sub(pattern,"",string)
    text1=re.sub(r"\([^()]*\)", '', clean)
    text=re.sub(r'\([^)]*\)',"",text1)
    return(text)


def save(leaders_per_country):
    out_file = open("myfile.json", "w")
    json.dump(leaders_per_country, out_file)
    out_file.close()
    f = open("myfile.json")
    leaders = json.load(f)
    print(leaders)
    f.close()

    
def get_leaders(session):
    root_url="https://country-leaders.herokuapp.com"
    status_url="status"
    endpoint="countries"
    cookie_url="cookie"
    leaders_url="leaders"
    cookie=requests.get(f"{root_url}/{cookie_url}")
    country_list=requests.get(f"{root_url}/{endpoint}",cookies=cookie.cookies)
    if country_list.status_code==403:
        cookie=requests.get(f"{root_url}/{cookie_url}")
        country_list=requests.get(f"{root_url}/{endpoint}",cookies=cookie.cookies)
    countries=country_list.json()
    leaders_per_country={}
    for country in countries:
        params={
        "country":country
        }
        leaders=requests.get(f"{root_url}/{leaders_url}",params=params,cookies=cookie.cookies)
        leaders_per_country[country] = leaders.json()
    #call to get_first_paragraph
    session=requests.session()
    leaders_per_country_updated={}
    for country, leaders in leaders_per_country.items():
        leaders_per_country_updated[country] = []
        for leader in leaders: 
            leader["Introduction"] = get_first_paragraph(leader["wikipedia_url"],session)
            leaders_per_country_updated[country].append(leader)
    return(leaders_per_country_updated)


session=requests.session()
leaders_per_country=get_leaders(session)
save(leaders_per_country)
